## データチェック用のnotebook

エラーが出たときにscpでファイルをローカルに転送してこのnotebookで細かくチェックする

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
from bs4 import BeautifulSoup
import jaconv
import csv
import re
import sys
import os
from datetime import datetime as dt
from lib import myfunc

In [2]:
## init parser
parser = EdinetXbrlParser()

In [3]:
# どっちがどっちか要確認
# 単体 or 連結のkeyの辞書
fs_dict = {"non-consolidated": "jpcrp_cor:NotesTaxEffectAccountingFinancialStatementsTextBlock"
    , "consolidated": "jpcrp_cor:NotesTaxEffectAccountingConsolidatedFinancialStatementsTextBlock"}

context_ref = "CurrentYearDuration"

In [4]:
### チェック用のファイルと使用するキー
# ファイル
xbrl_file_path = "./data/jpcrp030000-asr-001_E01859-000_2019-03-31_01_2019-06-27.xbrl"
# キー
usekey = fs_dict["consolidated"]

In [5]:
# 対象ファイルをパース
edinet_xbrl_object = parser.parse_file(xbrl_file_path)

In [6]:
# 会計基準を取得
acc_standard = edinet_xbrl_object.get_data_by_context_ref("jpdei_cor:AccountingStandardsDEI", "FilingDateInstant").get_value()
acc_standard

'Japan GAAP'

In [7]:
# 辞書のキーでカラム名を変える
if usekey == "consolidated":
    colname_tmp = "連結会計年度"
else:
    colname_tmp = "事業年度"

In [8]:
# テーブルの確認
table = myfunc.get_table(edinet_xbrl_object, usekey, context_ref)

# tableがない場合はスキップ
if table is None:
    # ToDo: loggingでlog出力したい
    print("There is no table.")

In [9]:
table_list = myfunc.table_to_list(table)
df = myfunc.list_to_pd(table_list)

[['', '', '前連結会計年度（2018年３月31日）', '', '', '当連結会計年度（2019年３月31日）', ''],
 ['', '', '', '（千円）', '', '', '', '（千円）', ''],
 ['\u3000\u3000\u3000繰延税金資産', '', '', '', '', '', '', '', ''],
 ['\u3000\u3000\u3000\u3000賞与引当金',
  '',
  '',
  '238,779',
  '',
  '',
  '',
  '239,248',
  ''],
 ['\u3000\u3000\u3000\u3000たな卸資産', '', '', '93,305', '', '', '', '101,486', ''],
 ['\u3000\u3000\u3000\u3000退職給付に係る負債',
  '',
  '',
  '1,411,245',
  '',
  '',
  '',
  '1,273,003',
  ''],
 ['\u3000\u3000\u3000\u3000役員退職慰労引当金',
  '',
  '',
  '37,437',
  '',
  '',
  '',
  '10,243',
  ''],
 ['\u3000\u3000\u3000\u3000工事損失引当金',
  '',
  '',
  '37,001',
  '',
  '',
  '',
  '326,900',
  ''],
 ['\u3000\u3000\u3000\u3000資産除去債務', '', '', '26,180', '', '', '', '25,926', ''],
 ['\u3000\u3000\u3000\u3000減損損失', '', '', '333,484', '', '', '', '291,239', ''],
 ['\u3000\u3000\u3000\u3000税務上の繰越欠損金（注）２',
  '',
  '',
  '485,586',
  '',
  '',
  '',
  '609,816',
  ''],
 ['\u3000\u3000\u3000\u3000その他', '', '', '204,153', '', '', '', '356,

In [10]:
df

,account,,前連結会計年度(2018年3月31日),前連結会計年度(2018年3月31日)_unit,,,当連結会計年度(2019年3月31日),当連結会計年度(2019年3月31日)_unit,
0,,,,（千円）,,,,（千円）,
1,繰延税金資産,,,,,,,,
2,賞与引当金,,,"238,779",,,,"239,248",
3,たな卸資産,,,"93,305",,,,"101,486",
4,退職給付に係る負債,,,"1,411,245",,,,"1,273,003",
5,役員退職慰労引当金,,,"37,437",,,,"10,243",
6,工事損失引当金,,,"37,001",,,,"326,900",
7,資産除去債務,,,"26,180",,,,"25,926",
8,減損損失,,,"333,484",,,,"291,239",
9,税務上の繰越欠損金(注)2,,,"485,586",,,,"609,816",


In [11]:
df.columns

Index(['account', '', '前連結会計年度(2018年3月31日)', '前連結会計年度(2018年3月31日)_unit', '',
       '', '当連結会計年度(2019年3月31日)', '当連結会計年度(2019年3月31日)_unit', ''],
      dtype='object')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 9 columns):
account                     19 non-null object
                            19 non-null object
前連結会計年度(2018年3月31日)         19 non-null object
前連結会計年度(2018年3月31日)_unit    19 non-null object
                            19 non-null object
                            19 non-null object
当連結会計年度(2019年3月31日)         19 non-null object
当連結会計年度(2019年3月31日)_unit    19 non-null object
                            19 non-null object
dtypes: object(9)
memory usage: 1.4+ KB


In [12]:
# # dfが6列の場合は単位のカラムの補完と値の数値変換を行う
# if len(df.columns) == 6:
#     df.loc[:, df.columns.str.contains("_unit")] = df.loc[:, df.columns.str.contains("_unit")].apply(myfunc.fill_unit)
#     # 値のカラム
#     collist_val = [i for i in df.columns[df.columns.str.contains(colname_tmp)] if re.search("^.*(?<!_unit)$", i)]
#     df.loc[:, collist_val] = df.loc[:, collist_val].applymap(myfunc.get_value)

# dfが4列の場合は単位の分離を行う
if len(df.columns) <= 4:
    # colname_tmpが含まれるカラムを取得
    colname_tmp_list = [i for i in df.columns if re.search(colname_tmp, i)]

    for col in colname_tmp_list:
        df = myfunc.sep_unit(df, col)

In [13]:
df

,account,前連結会計年度(2018年3月31日),前連結会計年度(2018年3月31日)_unit,,当連結会計年度(2019年3月31日),当連結会計年度(2019年3月31日)_unit,
0,繰延税金資産,,,,,,
1,税務上の繰越欠損金(注),"8,529",千円,,"12,266",千円,
2,退職給付に係る負債,"257,424",〃,,"265,303",〃,
3,棚卸資産評価損,"40,168",〃,,"38,628",〃,
4,賞与引当金,"41,588",〃,,"36,752",〃,
5,役員退職慰労引当金,"26,314",〃,,"27,637",〃,
6,減価償却費限度超過額,"19,686",〃,,"18,847",〃,
7,投資有価証券評価損,"10,617",〃,,"10,617",〃,
8,減損損失,"9,034",〃,,"8,437",〃,
9,資産除去債務,"4,311",〃,,"4,311",〃,


In [14]:
df.columns

Index(['account', '前連結会計年度(2018年3月31日)', '前連結会計年度(2018年3月31日)_unit', '',
       '当連結会計年度(2019年3月31日)', '当連結会計年度(2019年3月31日)_unit', ''],
      dtype='object')

In [15]:
# カラム名から期間の情報を抽出して別のカラムにし、カラム名からは期間を削除
df_output = myfunc.sep_period(df)

In [16]:
df_output

,account,prev_value,prev_value_unit,,cur_value,cur_value_unit,,cur_period,prev_period
0,繰延税金資産,,,,,,,2019年3月31日,2018年3月31日
1,税務上の繰越欠損金(注),"8,529",千円,,"12,266",千円,,2019年3月31日,2018年3月31日
2,退職給付に係る負債,"257,424",〃,,"265,303",〃,,2019年3月31日,2018年3月31日
3,棚卸資産評価損,"40,168",〃,,"38,628",〃,,2019年3月31日,2018年3月31日
4,賞与引当金,"41,588",〃,,"36,752",〃,,2019年3月31日,2018年3月31日
5,役員退職慰労引当金,"26,314",〃,,"27,637",〃,,2019年3月31日,2018年3月31日
6,減価償却費限度超過額,"19,686",〃,,"18,847",〃,,2019年3月31日,2018年3月31日
7,投資有価証券評価損,"10,617",〃,,"10,617",〃,,2019年3月31日,2018年3月31日
8,減損損失,"9,034",〃,,"8,437",〃,,2019年3月31日,2018年3月31日
9,資産除去債務,"4,311",〃,,"4,311",〃,,2019年3月31日,2018年3月31日


In [17]:
# 単位のカラムの補完と値の数値変換
df_output.loc[:, df_output.columns.str.contains("_unit")] = df_output.loc[:, df_output.columns.str.contains("_unit")].apply(myfunc.fill_unit)
# 値のカラムを数値にする
#df_output.loc[:, ["cur_value", "prev_value"]] = df_output.loc[:, ["cur_value", "prev_value"]].applymap(myfunc.get_value)
df_output["cur_value"] = df_output[["account", "cur_value"]].apply(lambda x: myfunc.get_value(x[0], x[1]), axis=1)
df_output["prev_value"] = df_output[["account", "prev_value"]].apply(lambda x: myfunc.get_value(x[0], x[1]), axis=1)

In [18]:
df_output

,account,prev_value,prev_value_unit,,cur_value,cur_value_unit,,cur_period,prev_period
0,繰延税金資産,NaN,,,NaN,,,2019年3月31日,2018年3月31日
1,税務上の繰越欠損金(注),8529.0,千円,,12266.0,千円,,2019年3月31日,2018年3月31日
2,退職給付に係る負債,257424.0,千円,,265303.0,千円,,2019年3月31日,2018年3月31日
3,棚卸資産評価損,40168.0,千円,,38628.0,千円,,2019年3月31日,2018年3月31日
4,賞与引当金,41588.0,千円,,36752.0,千円,,2019年3月31日,2018年3月31日
5,役員退職慰労引当金,26314.0,千円,,27637.0,千円,,2019年3月31日,2018年3月31日
6,減価償却費限度超過額,19686.0,千円,,18847.0,千円,,2019年3月31日,2018年3月31日
7,投資有価証券評価損,10617.0,千円,,10617.0,千円,,2019年3月31日,2018年3月31日
8,減損損失,9034.0,千円,,8437.0,千円,,2019年3月31日,2018年3月31日
9,資産除去債務,4311.0,千円,,4311.0,千円,,2019年3月31日,2018年3月31日
